In [ ]:
!pip install featuretools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.9/587.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.2/215.2 kB 13.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import numpy as np

# Create a mock customer DataFrame
customers_df = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5],
    'joined': pd.to_datetime(['2018-01-01', '2018-01-02', '2018-02-01', '2018-02-02', '2018-03-01']),
    'loves_ice_cream': [True, False, False, True, True],
    'favorite_number': [42, 28, 3, 8, 11]
})

# Create a mock transactions DataFrame
transactions_df = pd.DataFrame({
    'transaction_id': range(500),
    'customer_id': np.random.choice([1, 2, 3, 4, 5], 500),
    'amount': np.random.rand(500) * 100,
    'transaction_time': pd.date_range('2023-01-01', periods=500, freq='H')
})

In [ ]:
customers_df

,customer_id,joined,loves_ice_cream,favorite_number
0,1,2018-01-01,True,42
1,2,2018-01-02,False,28
2,3,2018-02-01,False,3
3,4,2018-02-02,True,8
4,5,2018-03-01,True,11


In [ ]:
transactions_df

,transaction_id,customer_id,amount,transaction_time
0,0,4,93.518231,2023-01-01 00:00:00
1,1,4,12.163922,2023-01-01 01:00:00
2,2,3,16.139716,2023-01-01 02:00:00
3,3,2,46.105018,2023-01-01 03:00:00
4,4,1,88.193475,2023-01-01 04:00:00
...,...,...,...,...
495,495,2,81.521830,2023-01-21 15:00:00
496,496,3,60.985661,2023-01-21 16:00:00
497,497,4,66.806382,2023-01-21 17:00:00
498,498,1,20.833348,2023-01-21 18:00:00


In [ ]:
import featuretools as ft

# Create a new entityset
es = ft.EntitySet(id='customer_data')

# Add the customer dataframe as an entity
es = es.add_dataframe(dataframe_name='customers', dataframe=customers_df, index='customer_id')

# Add the transactions dataframe as an entity and specify the relationship
es = es.add_dataframe(dataframe_name='transactions', dataframe=transactions_df,
                      index='transaction_id', time_index='transaction_time')

es = es.add_relationship('customers', 'customer_id', 'transactions', 'customer_id')

In [ ]:
# Automatically generate features
feature_matrix, feature_defs = ft.dfs(entityset=es, target_dataframe_name='customers')

/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x786e38b13d00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function max at 0x786e38b13400> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function min at 0x786e38b13520> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable wil

In [ ]:
feature_matrix.head()

,loves_ice_cream,favorite_number,COUNT(transactions),MAX(transactions.amount),MEAN(transactions.amount),MIN(transactions.amount),SKEW(transactions.amount),STD(transactions.amount),SUM(transactions.amount),DAY(joined),...,WEEKDAY(joined),YEAR(joined),MODE(transactions.DAY(transaction_time)),MODE(transactions.MONTH(transaction_time)),MODE(transactions.WEEKDAY(transaction_time)),MODE(transactions.YEAR(transaction_time)),NUM_UNIQUE(transactions.DAY(transaction_time)),NUM_UNIQUE(transactions.MONTH(transaction_time)),NUM_UNIQUE(transactions.WEEKDAY(transaction_time)),NUM_UNIQUE(transactions.YEAR(transaction_time))
customer_id,,,,,,,,,,,,,,,,,,,,,
1,True,42,105,98.818875,46.567330,3.510532,0.201058,27.531262,4889.569632,1,...,0,2018,13,1,6,2023,20,1,7,1
2,False,28,121,98.791077,46.131805,0.182750,0.232723,28.262803,5581.948353,2,...,1,2018,5,1,0,2023,21,1,7,1
3,False,3,96,98.914812,50.559248,0.396600,0.029818,28.294532,4853.687795,1,...,3,2018,3,1,1,2023,21,1,7,1
4,True,8,86,99.235374,48.741681,1.651393,0.111845,26.797198,4191.784563,2,...,4,2018,18,1,5,2023,21,1,7,1
5,True,11,92,99.298725,52.280196,0.004280,-0.095157,29.418096,4809.778059,1,...,3,2018,2,1,0,2023,20,1,7,1


In [ ]:
feature_matrix.columns

Index(['loves_ice_cream', 'favorite_number', 'COUNT(transactions)',
       'MAX(transactions.amount)', 'MEAN(transactions.amount)',
       'MIN(transactions.amount)', 'SKEW(transactions.amount)',
       'STD(transactions.amount)', 'SUM(transactions.amount)', 'DAY(joined)',
       'MONTH(joined)', 'WEEKDAY(joined)', 'YEAR(joined)',
       'MODE(transactions.DAY(transaction_time))',
       'MODE(transactions.MONTH(transaction_time))',
       'MODE(transactions.WEEKDAY(transaction_time))',
       'MODE(transactions.YEAR(transaction_time))',
       'NUM_UNIQUE(transactions.DAY(transaction_time))',
       'NUM_UNIQUE(transactions.MONTH(transaction_time))',
       'NUM_UNIQUE(transactions.WEEKDAY(transaction_time))',
       'NUM_UNIQUE(transactions.YEAR(transaction_time))'],
      dtype='object')